## ReMKiT1D input generator - 0D hydrogen CRM using CRM modelbound data and term generator functionality

A feature that adds flexibility and convenience to ReMKiT1D is associating term generators with custom models in addition to specifying user-generated terms. This notebook shows how to use the CRM modelbound data class in ReMKiT1D together with the CRM density evolution term generator to build a hydrogen CRM with inbuilt data. 

Initial data are designed to reproduce Figure 8 in Colonna et al. (Spectrochimica Acta Part B 56 2001 587᎐598) or to show convergence to Saha-Boltzmann for opaque plasmas.

This notebook corresponds to the two fluid test performed in Section 5.3. in the ReMKiT1D code paper.

In [ ]:
import numpy as np
import xarray as xr
import holoviews as hv
import matplotlib.pyplot as plt
import matplotlib as mpl
from holoviews import opts
import panel as pn

import sys
sys.path.append('../')
from RMK_support import RKWrapper ,Grid
import RMK_support.simple_containers as sc
import RMK_support.IO_support as io
import RMK_support.dashboard_support as ds
import RMK_support.common_models as cm
import RMK_support.sk_normalization as skn
import RMK_support.crm_support as crm # This is the main python module for crm construction


### Some useful constants

In [ ]:
heavySpeciesMass = 2.014 #in amus
hPlanck = 6.62607004e-34
elMass =  9.10938e-31
elCharge = 1.60218e-19


### Wrapper initialization

In [ ]:
rk = RKWrapper()

### Filepaths

In [ ]:
rk.jsonFilepath = "./config.json" # Default value
hdf5Filepath = "./RMKOutput/RMK_CRM_example/"
rk.setHDF5Path(hdf5Filepath) 

### Normalization setup

In [ ]:
rk.setNormDensity(1.0e19)
rk.setNormTemperature(10.0)
rk.setNormRefZ(1.0)

tempNorm = rk.normalization["eVTemperature"] 
densNorm = rk.normalization["density"]
timeNorm = skn.collTimeei(tempNorm,densNorm,rk.normalization["referenceIonZ"])


### Grid setup

In [ ]:
xGrid = np.ones(1) # 0D
# Need a non-trivial velocity grid for <sigma v> integrals
vGrid = np.logspace(-2,0,80) #In normalized velocity - default normalization is thermal velocity sqrt(m_e * k * T_e/2)

lMax = 0 
gridObj = Grid(xGrid,vGrid,lMax,interpretXGridAsWidths=True ,interpretVGridAsWidths=True)

In [ ]:
# Add the grid to the wrapper
rk.grid=gridObj

### Species initialization 

NOTE: The CRM density evolution generator assumes that all species it evolves have a density as their first associated variable and that it is an implicit and fluid variable. Also, the prebuilt Janev data requires that neutral IDs correspond to principle quantum numbers of excited states.

In [ ]:
electronSpecies = sc.Species("e",0,associatedVars=["ne"]) 
ionSpecies = sc.Species("D+",-1,atomicA=heavySpeciesMass,charge=1.0,associatedVars=["ni"])

rk.addSpecies("e",0,associatedVars=["ne"])
rk.addSpecies("D+",-1,atomicA=heavySpeciesMass,charge=1.0,associatedVars=["ni"])

numNeutrals=25
neutralDensList = ["n"+str(i) for i in range(1,numNeutrals+1)] # List of neutral density names

for neutral in neutralDensList:
    rk.addSpecies("D"+neutral[1:],int(neutral[1:]),heavySpeciesMass,associatedVars=[neutral])

### Variable initialization

In [ ]:
T0 = 1.72 # Temperature in eV approx 20000K corresponding to case 1 in Colonna et al.
T = T0*np.ones(gridObj.numX())/tempNorm

rk.addVar("T",T,units='$10eV$',isDerived=True,outputVar=True)
rk.addVar("time",isScalar=True,isDerived=True,outputVar=True)

# 10% ionization fixed initial densities with no excited states
ne = 0.9
n1 = 0.1

ntot = ne + n1 #total density in case we want to calculate Saha-Boltzmann distribution

reprColonna = True # Set to true to approximately reproduce 10^-8 line in figure 8 of Colonna et al. If false will run to equilibrium at lower density
fixedID0 =  None # No fixed initial ionization degree
if reprColonna:
    ntot = 733893.9 # Density corresponding to approximately 1atm of pressure at 1000K
    fixedID0 = 1e-3


stateTempInit = T0/2 # (Saha-)Boltzmann temperature corresponding to case 1 in Colonna et al.
neutDensSBInit = [dens/densNorm for dens in crm.hydrogenSahaBoltzmann(numNeutrals,stateTempInit,ntot*densNorm,fixedIonizationDegree=fixedID0)]

initialSahaBoltzmann = True # Set to true for initial (Saha-)Boltzmann condition
if initialSahaBoltzmann: 
    n = neutDensSBInit[0]*np.ones(gridObj.numX())  
else:
    n = ne*np.ones(gridObj.numX())

# We need a distribution function to calculate rates from cross-sections built into the code
f = np.zeros([gridObj.numX(),gridObj.numH(),gridObj.numV()])
for i in range(gridObj.numX()):
    f[i,gridObj.getH(0)-1,:] = (T[i]*np.pi)**(-1.5) * n[i] * np.exp(-gridObj.vGrid**2/T[i])
if reprColonna:
    rk.addVar("f",f,isDerived=True,isDistribution=True,outputVar=True)
    
else:
    rk.addVar("f",f,isDerived=True,isDistribution=True,derivationRule=sc.derivationRule("maxwellianDistribution",["T","ne"]),outputVar=True)
    
rk.addVar("ne",n,units='$10^{19} m^{-3}$',outputVar=True)
rk.addVar("ni",n,units='$10^{19} m^{-3}$',outputVar=True)

if initialSahaBoltzmann:
    for i in range(1,numNeutrals+1):
        rk.addVar(neutralDensList[i-1],neutDensSBInit[i]*np.ones(gridObj.numX()),units='$10^{19} m^{-3}$',outputVar=True)
else:
    rk.addVar("n1",n1*np.ones(gridObj.numX()),units='$10^{19} m^{-3}$',outputVar=True)
    for i in range(2,numNeutrals+1):
        rk.addVar(neutralDensList[i-1],units='$10^{19} m^{-3}$',outputVar=True)


# Calculate expected Saha-Boltzmann at T0
neutDensSB = [dens/densNorm for dens in crm.hydrogenSahaBoltzmann(numNeutrals,T0,ntot*densNorm)]

### External libraries

#### MPI

Single processor setup.

In [ ]:
numProcsX = 1 # Number of processes in x direction
numProcsH = 1 # Number of processes in harmonic direction
numProcs = numProcsH*numProcsX
haloWidth = 1 # Halo width in cells
rk.setMPIData(numProcsX,numProcsH,haloWidth)

#### PETSc

Defaults.

#### HDF5

Output variables set when adding variables

### Models

Here only a single model is added, and is constructed using the CRM term generator and modelbound CRM data

#### Generating modelbound data based on inbuilt cross-sections

The ModelboundCRMData object simplifies the construction of CRM modelbound data using the crm_support module. See the module documentation for details on how to add different types of currently supported transition objects.

NOTE: Inbuilt hydrogen cross-sections in ReMKiT1D are based on Janev fits. Transitions which use Janev cross-sections and rates can be added using the addJanevTransitions function from the crm module.

NOTE: Rates of modelbound transitions can be accessed as named modelbound row data in the form "rate{ID}index{transIndex}" where ID is 0 for the reaction (particle) rate, 1 for momentum (not always supported!) and 2 for energy rates. transIndex is the index of the transition in the ModelboundCRMData.transitionTags list. For standard names of inbuilt transitions see crm_support module.

In [ ]:
includeRadiativeProcesses = reprColonna # Should be included for Colonna Figure 8 reproduction and turned off for Saha-Boltzmann convergence
#Set the list of included Janev transitions
includedJanevTransitions = ["ex","deex","ion","recomb3b"] 
if includeRadiativeProcesses:
    includedJanevTransitions.append("recombRad") #Add radiative recombination if radiative processes are included
mbData = crm.ModelboundCRMData()
crm.addJanevTransitionsToCRMData(mbData,numNeutrals,tempNorm,"f","T",detailedBalanceCSPriority=1,processes=includedJanevTransitions)

#### Reading and adding NIST data for spontaneous transition probabilities

Hydrogen transition probabilities are included in the Aki.csv file, and have been taken from the NIST database.

In [ ]:

"""
Kramida, A., Ralchenko, Yu., Reader, J., and NIST ASD Team (2021). NIST Atomic Spectra Database (ver. 5.9), [Online]. Available: https://physics.nist.gov/asd [2022, May 13]. National Institute of Standards and Technology, Gaithersburg, MD. DOI: https://doi.org/10.18434/T4W30F
"""

spontTransDict = crm.readNISTAkiCSV("../data/Aki.csv")

Adding hydrogen spontaneous emission transitions can be done using the following function.

In [ ]:
if includeRadiativeProcesses:
    crm.addHSpontaneousEmissionToCRMData(mbData,spontTransDict,min(numNeutrals,20),min(numNeutrals,20),timeNorm,tempNorm) #NIST data only has a full transition list for n<=20

#### Creating the CRM model object

The CRM model is constructed by specifying the modelbound data (defined above) and adding a term generator that can interpret that data and create a collisional-radiative model from it. 

Term generators are sets of rules used to automate term construction. The CRM term generator uses the modelbound CRM data of its host model, identifies which transitions produce a change in the populations of evolved species, and generates corresponding source and sink terms for each species. 

In [ ]:
#CRM model
 
#Adding the model tag to tag list
modelTag = "CRMmodel"

#Initializing model
crmModel = sc.CustomModel(modelTag=modelTag)

crmModel.setModelboundData(mbData.dict())

#Add term generator responsible for buildling CRM model
crmTermGenerator = crm.termGeneratorCRM()

crmModel.addTermGenerator("crmTermGen",crmTermGenerator)

#Add model to wrapper

rk.addModel(crmModel.dict())


### Integrator and timestep options

Simple single step backwards Euler integration

In [ ]:
integrator = sc.picardBDEIntegrator(absTol=100.0,convergenceVars=["f","n1"]) 

rk.addIntegrator("BE",integrator)

Set initial timestep length and numbers of allowed implicit and general groups

In [ ]:
initialTimestep = 1e5 # Large timestep to obtain steady state
if reprColonna:
    initialTimestep = 1e-4 # Small timestep to resolve evolution for Colonna Fig 8 reproduction

rk.setIntegratorGlobalData(1,1,initialTimestep) 

In [ ]:
bdeStep = sc.IntegrationStep("BE")

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("BE1",bdeStep.dict())

#### Timeloop options

In [ ]:
rk.setFixedNumTimesteps(1500)
rk.setFixedStepOutput(100)

### Create config file

In [ ]:
rk.writeConfigFile()

### Data analysis


In [ ]:
numFiles = 15

#### Loading data

Set loadpath to ReMKiT1D directory

In [ ]:
loadpath = hdf5Filepath
loadFilenames = [loadpath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]

In [ ]:
loadedData = io.loadFromHDF5(rk.varCont,filepaths=loadFilenames)
loadedData

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=80,dpi=150)

#### Compare final state densities with a Saha-Boltzmann equilibrium

If reprColonna is true will compare to the initial distribution, otherwise will compare to the expected Saha-Boltzmann distribution at the electron temperature.

In [ ]:
# Neutral state densities extracted from dataset
neutralDens1 = [loadedData[neutralDensList[i]][-1,0] for i in range(numNeutrals)]

# Excited state energies
stateEnergies = [13.6*(1-1/(i+1)**2) for i in range(numNeutrals)]

if reprColonna:
    coords1 = [(stateEnergies[i],neutralDens1[i]/((i+1)**2)/(neutralDens1[0])) for i in range(numNeutrals)] #Degeneracy weighted densities in final timestep
    
    coords2 = [(stateEnergies[i],neutDensSBInit[i+1]/((i+1)**2*neutDensSBInit[1]) )for i in range(numNeutrals)] #Initial Saha-Boltzmann densities (weighted)
else:
    coords1 = [(stateEnergies[i],neutralDens1[i]/(2*(i+1)**2)) for i in range(numNeutrals)] #Degeneracy weighted densities in final timestep
    coords2 = [(stateEnergies[i],neutDensSB[i+1]/(2*(i+1)**2)) for i in range(numNeutrals)] #Expected Saha-Boltzmann densities (weighted)
    
if reprColonna: 
    label2 = 't = 0'
    label1 = f't = {loadedData.coords["time"].values[-1]*timeNorm:.2e}s'
else:
    label1 = 'ReMKiT1D'
    label2 = 'Saha-Boltzmann'
    
curve2 = hv.Curve(coords2,label=label2).opts(color="r")
if reprColonna:
    curve1 = hv.Curve(coords1,label=label1).opts(color="k",linestyle='--')
else:
    curve1 = hv.Scatter(coords1,label=label1).opts(marker="x",color="k",s=15.0)
    
curve = curve2*curve1
curve.opts(logy=True,xlabel="E [eV]",ylabel='$n_i/g_i$',aspect=0.5)


#### Produce paper plots

In [ ]:
if reprColonna:
    hv.save(curve.opts(logy=True,xlabel="E [eV]",ylabel='$n_i/g_i$',aspect=0.5),"colonna_comp.pdf",dpi=144)
else:
    hv.save(curve.opts(logy=True,xlabel="E [eV]",ylabel='$n_i/g_i$',aspect=0.5),"sb_comp.pdf",dpi=144)